## Generación base de datos el Parque Nacional Cerro Saroche

**PROYECTO:** SISTEMA PARA EL SEGUIMIENTO DE ECOSISTEMAS VENEZOLANOS \
**AUTOR:** Javier Martinez

In [1]:
import rioxarray 
import xarray

import geopandas
from pyproj.crs import CRS

import pandas as pd

### Proyección

In [2]:
precipitacion_crs = CRS.from_wkt('GEOGCS["Coordinate System imported from GRIB file",DATUM["unnamed",SPHEROID["Sphere",6367470,0]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST]]')

### Polígonos

In [3]:
park_poligono = geopandas.read_file('./cerro_saroche/poligono_cerro_saroche/cerro_saroche.shp')
parque_rectangulo = geopandas.read_file('./cerro_saroche/poligono_cerro_saroche/rectangulo_cerro_saroche.shp')

### Paths

In [4]:
path_precipitacion = './cerro_saroche/cdsapi/precipitacion_regrilla.nc'
path_elevacion = './cerro_saroche/SRTMGL3/elevacion_regrilla.nc'
path_ndvi = './cerro_saroche/ndvi/ndvi_regrilla.nc'

### Precipitación

In [5]:
precipitacion_rds = rioxarray.open_rasterio(path_precipitacion, masked=True)
precipitacion_rds

<xarray.DataArray 'precipitacion_mm' (time: 628, y: 3, x: 5)>
array([[[0.913065, 0.958915, ..., 1.095035, 1.203287],
        [1.033501, 1.10399 , ..., 1.212156, 1.276562],
        [1.229499, 1.29802 , ..., 1.328971, 1.300588]],

       [[0.081278, 0.112999, ..., 0.220146, 0.303084],
        [0.07641 , 0.100975, ..., 0.192905, 0.258515],
        [0.106268, 0.12937 , ..., 0.207852, 0.246627]],

       ...,

       [[0.418985, 0.475595, ..., 0.894255, 1.082145],
        [0.505286, 0.46067 , ..., 0.879318, 1.063786],
        [0.564628, 0.429824, ..., 0.673022, 0.842776]],

       [[3.615426, 3.626734, ..., 3.381902, 3.350183],
        [3.842396, 3.955119, ..., 3.782641, 3.754916],
        [4.20996 , 4.2078  , ..., 4.122327, 4.089661]]])
Coordinates:
  * time         (time) int64 719163 719194 719222 ... 738187 738215 738246
  * x            (x) float64 -69.83 -69.73 -69.63 -69.53 -69.43
  * y            (y) float64 10.31 10.21 10.11
    spatial_ref  int64 0
Attributes:
    add_offset:             0.0
    GRIB_COMMENT:           Total precipitation [m]
    GRIB_ELEMENT:           TP
    GRIB_FORECAST_SECONDS:  86400
    GRIB_REF_TIME:          0
    GRIB_SHORT_NAME:        0-SFC
    GRIB_UNIT:              [m]
    GRIB_VALID_TIME:        86400
    long_name:              0[-] SFC (Ground or water surface)
    scale_factor:           1.0

In [6]:
def id_point_format(x,y):
  """
  Funcion para identificar el id del centroide
  """

  #--
  if x == -69.43 and y==10.11:
    id_point = 15
  elif x == -69.53 and y==10.11:
    id_point = 14
  elif x == -69.63 and y==10.11:
    id_point = 13
  elif x == -69.73 and y==10.11:
    id_point = 12
  elif x == -69.83 and y==10.11:
    id_point = 11
  #--
  elif x == -69.43 and y==10.21:
    id_point = 10
  elif x == -69.53 and y==10.21:
    id_point = 9
  elif x == -69.63 and y==10.21:
    id_point = 8
  elif x == -69.73 and y==10.21:
    id_point = 7
  elif x == -69.83 and y==10.21:
    id_point = 6
  #--
  elif x == -69.43 and y==10.31:
    id_point = 5
  elif x == -69.53 and y==10.31:
    id_point = 4
  elif x == -69.63 and y==10.31:
    id_point = 3
  elif x == -69.73 and y==10.31:
    id_point = 2
  elif x == -69.83 and y==10.31:
    id_point = 1
  else:
    id_point = None

  return id_point

In [7]:
columns_precipitacion = ['time',	'x',	'y', 'precipitacion_mm']
pd_precipitacion = precipitacion_rds.to_dataframe()\
                                    .reset_index()[columns_precipitacion]

pd_precipitacion['time'] = pd_precipitacion['time'].astype(int)
pd_precipitacion['x'] = pd_precipitacion['x'].astype(float).round(6)
pd_precipitacion['y'] = pd_precipitacion['y'].astype(float).round(6)
pd_precipitacion['id_point'] = pd_precipitacion[['x','y']].round(2).apply(lambda x: id_point_format(x=x.x,y=x.y), 1 ).astype(int)
pd_precipitacion['precipitacion_mm'] = pd_precipitacion['precipitacion_mm'].astype(float)

pd_precipitacion = pd_precipitacion.dropna()

pd_precipitacion.head(10)

,time,x,y,precipitacion_mm,id_point
0,719163,-69.83,10.31,0.913065,1
1,719163,-69.73,10.31,0.958915,2
2,719163,-69.63,10.31,1.026073,3
3,719163,-69.53,10.31,1.095035,4
4,719163,-69.43,10.31,1.203287,5
5,719163,-69.83,10.21,1.033501,6
6,719163,-69.73,10.21,1.103990,7
7,719163,-69.63,10.21,1.159936,8
8,719163,-69.53,10.21,1.212156,9
9,719163,-69.43,10.21,1.276562,10


### Elevación

In [8]:
elevacion_rds = rioxarray.open_rasterio(path_elevacion, masked=True)
elevacion_rds

<xarray.Dataset>
Dimensions:            (y: 3, x: 5, band: 1)
Coordinates:
  * y                  (y) float64 10.31 10.21 10.11
  * x                  (x) float64 -69.83 -69.73 -69.63 -69.53 -69.43
  * band               (band) int64 1
    spatial_ref        int64 0
Data variables:
    elevacion_media    (band, y, x) float32 ...
    elevacion_mediana  (band, y, x) float32 ...
    elevacion_maxima   (band, y, x) float32 ...
Attributes:
    Conventions:  CF-1.7
    history:      2022-07-31T11:48 GRIB to CDM+CF via cfgrib-0.9.10.1/ecCodes...

In [9]:
columns_elevacion = ['x',	'y', 'elevacion_media','elevacion_mediana','elevacion_maxima']
pd_elevacion = elevacion_rds.to_dataframe()\
                            .reset_index()[columns_elevacion]

pd_elevacion['x'] = pd_elevacion['x'].astype(float).round(6)
pd_elevacion['y'] = pd_elevacion['y'].astype(float).round(6)
pd_elevacion['elevacion_media'] = pd_elevacion['elevacion_media'].astype(float)
pd_elevacion['elevacion_mediana'] = pd_elevacion['elevacion_mediana'].astype(float)
pd_elevacion['elevacion_maxima'] = pd_elevacion['elevacion_maxima'].astype(float)

pd_elevacion.head(10)

,x,y,elevacion_media,elevacion_mediana,elevacion_maxima
0,-69.83,10.31,508.541046,491.0,921.0
1,-69.73,10.31,625.942932,614.0,1000.0
2,-69.63,10.31,731.954834,731.0,1025.0
3,-69.53,10.31,761.129150,737.0,1103.0
4,-69.43,10.31,726.967285,709.0,1202.0
5,-69.83,10.21,604.697083,558.0,1151.0
6,-69.73,10.21,652.016235,636.0,1114.0
7,-69.63,10.21,850.282715,839.0,1161.0
8,-69.53,10.21,769.925049,764.0,1025.0
9,-69.43,10.21,919.076721,895.0,1321.0


### NDVI

In [10]:
ndvi_rds = rioxarray.open_rasterio(path_ndvi, masked=True)
ndvi_rds

<xarray.Dataset>
Dimensions:       (y: 3, x: 6, time: 112)
Coordinates:
  * y             (y) float64 10.31 10.21 10.11
  * x             (x) float64 -69.93 -69.83 -69.73 -69.63 -69.53 -69.43
  * time          (time) int64 734503 734534 734563 ... 738246 738276 738307
    spatial_ref   int64 0
Data variables:
    ndvi_media    (time, y, x) float32 ...
    ndvi_mediana  (time, y, x) float32 ...
    ndvi_maxima   (time, y, x) float32 ...
Attributes:
    Conventions:  CF-1.7
    history:      2022-07-31T11:49 GRIB to CDM+CF via cfgrib-0.9.10.1/ecCodes...

In [11]:
columns_ndvi = ['time', 'x',	'y', 'ndvi_media','ndvi_mediana','ndvi_maxima']
pd_ndvi = ndvi_rds.to_dataframe()\
                  .reset_index()[columns_ndvi]

pd_ndvi['time'] = pd_ndvi['time'].astype(int)
pd_ndvi['x'] = pd_ndvi['x'].astype(float).round(6)
pd_ndvi['y'] = pd_ndvi['y'].astype(float).round(6)
pd_ndvi['ndvi_media'] = pd_ndvi['ndvi_media'].astype(float)
pd_ndvi['ndvi_mediana'] = pd_ndvi['ndvi_mediana'].astype(float)
pd_ndvi['ndvi_maxima'] = pd_ndvi['ndvi_maxima'].astype(float)

pd_ndvi.head(10)

,time,x,y,ndvi_media,ndvi_mediana,ndvi_maxima
0,734503,-69.93,10.31,NaN,NaN,NaN
1,734534,-69.93,10.31,NaN,NaN,NaN
2,734563,-69.93,10.31,NaN,NaN,NaN
3,734594,-69.93,10.31,NaN,NaN,NaN
4,734624,-69.93,10.31,NaN,NaN,NaN
5,734655,-69.93,10.31,NaN,NaN,NaN
6,734685,-69.93,10.31,NaN,NaN,NaN
7,734716,-69.93,10.31,NaN,NaN,NaN
8,734747,-69.93,10.31,NaN,NaN,NaN
9,734777,-69.93,10.31,NaN,NaN,NaN


#### Integrando Bases

In [12]:
from datetime import datetime

pd_integracion = pd.merge(pd_precipitacion, pd_ndvi, on = ['time','x','y'], how='left')
pd_integracion = pd.concat([ pd_integracion, pd_ndvi[pd_ndvi.time > pd_precipitacion.time.max()] ])\
                   .merge(pd_elevacion, on = ['x','y'], how='left')\
                   .rename(columns={"x": "longitud", "y": "latitud"})

pd_integracion = pd_integracion[pd_integracion.id_point.notna()]
pd_integracion['id_point'] = pd_integracion['id_point'].astype(int)
pd_integracion['time_actualizacion'] = int(datetime.today().toordinal())

pd_integracion.head(10)

,time,longitud,latitud,precipitacion_mm,id_point,ndvi_media,ndvi_mediana,ndvi_maxima,elevacion_media,elevacion_mediana,elevacion_maxima,time_actualizacion
0,719163,-69.83,10.31,0.913065,1,NaN,NaN,NaN,508.541046,491.0,921.0,738367
1,719163,-69.73,10.31,0.958915,2,NaN,NaN,NaN,625.942932,614.0,1000.0,738367
2,719163,-69.63,10.31,1.026073,3,NaN,NaN,NaN,731.954834,731.0,1025.0,738367
3,719163,-69.53,10.31,1.095035,4,NaN,NaN,NaN,761.129150,737.0,1103.0,738367
4,719163,-69.43,10.31,1.203287,5,NaN,NaN,NaN,726.967285,709.0,1202.0,738367
5,719163,-69.83,10.21,1.033501,6,NaN,NaN,NaN,604.697083,558.0,1151.0,738367
6,719163,-69.73,10.21,1.103990,7,NaN,NaN,NaN,652.016235,636.0,1114.0,738367
7,719163,-69.63,10.21,1.159936,8,NaN,NaN,NaN,850.282715,839.0,1161.0,738367
8,719163,-69.53,10.21,1.212156,9,NaN,NaN,NaN,769.925049,764.0,1025.0,738367
9,719163,-69.43,10.21,1.276562,10,NaN,NaN,NaN,919.076721,895.0,1321.0,738367


In [13]:
pd_integracion.groupby(['longitud','latitud']).count()

time  precipitacion_mm  id_point  ndvi_media  ndvi_mediana  \
longitud latitud                                                               
-69.83   10.11     628               628       628           0             0   
         10.21     628               628       628         110           110   
         10.31     628               628       628         110           110   
-69.73   10.11     628               628       628         110           110   
         10.21     628               628       628         110           110   
         10.31     628               628       628         110           110   
-69.63   10.11     628               628       628         110           110   
         10.21     628               628       628         110           110   
         10.31     628               628       628           0             0   
-69.53   10.11     628               628       628         110           110   
         10.21     628               628       628         110           110   
         10.31     628               628       628           0             0   
-69.43   10.11     628               628       628         110           110   
         10.21     628               628       628         110           110   
         10.31     628               628       628           0             0   

                  ndvi_maxima  elevacion_media  elevacion_mediana  \
longitud latitud                                                    
-69.83   10.11              0              628                628   
         10.21            110              628                628   
         10.31            110              628                628   
-69.73   10.11            110              628                628   
         10.21            110              628                628   
         10.31            110              628                628   
-69.63   10.11            110              628                628   
         10.21            110              628                628   
         10.31              0              628                628   
-69.53   10.11            110              628                628   
         10.21            110              628                628   
         10.31              0              628                628   
-69.43   10.11            110              628                628   
         10.21            110              628                628   
         10.31              0              628                628   

                  elevacion_maxima  time_actualizacion  
longitud latitud                                        
-69.83   10.11                 628                 628  
         10.21                 628                 628  
         10.31                 628                 628  
-69.73   10.11                 628                 628  
         10.21                 628                 628  
         10.31                 628                 628  
-69.63   10.11                 628                 628  
         10.21                 628                 628  
         10.31                 628                 628  
-69.53   10.11                 628                 628  
         10.21                 628                 628  
         10.31                 628                 628  
-69.43   10.11                 628                 628  
         10.21                 628                 628  
         10.31                 628                 628

In [14]:
import json

json_data = '{ "data":' + pd_integracion.to_json(orient="records") +\
                ',"park" : "cerro_saroche"' + "}"

with open('./cerro_saroche/data/json_data.json', 'w') as outfile:
    outfile.write(json_data)